# Calibration

Three methods are compared
1. AR6 WGI Chapter 7
2. Smith et al. (2021, JGR)
3. the present study with MCE-2l

Input
- `dataout/cmip6_normalized_1.csv`
    Normalized CMIP6 data from `010_cmip6_preprocess.ipynb`

Output
- `dataout/parms_calib.csv`
    Calibrated parameters harmonized across the three methods

In [1]:
import sys
import numpy as np
import pandas as pd
import json

In [2]:
# mce is available in https://github.com/tsutsui1872/mce 
sys.path.append('../mce')

In [3]:
from mce.core.climate_parms import ParmEstimate

In [4]:
from src.util import RetrieveGitHub
from src.tlm import ebm_to_irm, irm_to_ebm, add_ecs_tcr

## AR6 WGI Chapter 7

https://github.com/IPCC-WG1/Chapter-7

In [5]:
owner = 'IPCC-WG1'
repo = 'Chapter-7'
repo_ch7 = RetrieveGitHub(owner, repo, './datain')

In [6]:
path = repo_ch7.retrieve('data_input/tunings/cmip6_twolayer_tuning_params.json')

[2024-07-05 14:08:39 src.util] INFO:Use local file datain/IPCC-WG1/Chapter-7/data_input/tunings/cmip6_twolayer_tuning_params.json retrieved from https://github.com/IPCC-WG1/Chapter-7/raw/main/data_input/tunings/cmip6_twolayer_tuning_params.json on 2024-06-11


In [7]:
with path.open() as f1:
    contents = json.load(f1)

In [8]:
df = pd.concat({
    name: pd.DataFrame(contents[name]['model_data']).rename_axis('Dataset')
    for name in contents
}, names=['Parameter']).rename_axis(columns='Model').unstack('Parameter')
df

Model               EBM-1                                              \
Parameter             q4x      lamg        t4x       cdeep       cmix   
Dataset                                                                 
ACCESS-CM2       6.808006 -0.726863   9.366281   80.818713   7.627664   
ACCESS-ESM1-5    5.631650 -0.723839   7.780256   82.664315   6.914426   
AWI-CM-1-1-MR    7.275144 -1.151275   6.319207   50.868340   7.183540   
BCC-CSM2-MR      6.142330 -1.012061   6.069129   62.712983   5.063998   
BCC-ESM1         5.959903 -0.909903   6.550045   85.579490   7.953748   
CAMS-CSM1-0      8.549076 -1.860472   4.595111   52.081571   9.088186   
CESM2            6.473119 -0.625834  10.343195   68.393353   6.230418   
CESM2-FV2        5.798651 -0.569484  10.182290   82.204309   5.411523   
CESM2-WACCM      6.541041 -0.693532   9.431497   80.763464   6.783822   
CESM2-WACCM-FV2  5.965830 -0.633805   9.412718   98.636793   6.871732   
CNRM-CM6-1       7.319595 -0.749513   9.765795  146.059027   7.601150   
CNRM-CM6-1-HR    7.993945 -0.926194   8.630966  111.104775   8.946100   
CNRM-ESM2-1      5.979598 -0.627586   9.527927  100.845257   7.707239   
CanESM5          7.335038 -0.649509  11.293210   78.841943   8.082067   
E3SM-1-0         6.679023 -0.630884  10.586765   39.521636   7.531479   
FGOALS-f3-L      8.269494 -1.382010   5.983673   68.779075   7.271029   
FGOALS-g3        7.187004 -1.251138   5.744372   87.834040   7.237159   
GFDL-CM4         6.366342 -0.817647   7.786175   80.101167   6.160642   
GFDL-ESM4        7.605952 -1.439431   5.284000  162.681383   8.544148   
GISS-E2-1-G      8.029714 -1.490534   5.387139  134.279640   7.258398   
GISS-E2-1-H      7.038649 -1.128809   6.235466   81.092168   8.307487   
GISS-E2-2-G      7.305560 -1.504747   4.855009  644.074491   9.031724   
HadGEM3-GC31-LL  6.980813 -0.630032  11.080095   72.133047   7.526180   
HadGEM3-GC31-MM  7.108620 -0.652875  10.888180   71.465635   8.249073   
IITM-ESM         9.153635 -1.941729   4.714166  160.809815   9.033385   
INM-CM5-0        5.897931 -1.539974   3.829889   44.148149   7.845964   
IPSL-CM6A-LR     6.388994 -0.626867  10.191944   85.730639   6.485219   
MIROC-ES2L       8.230595 -1.547334   5.319212  184.485403  10.674440   
MIROC6           7.357368 -1.404636   5.237919  183.591183   8.524334   
MPI-ESM1-2-HR    7.199895 -1.201391   5.992966   84.691465   7.335933   
MRI-ESM2-0       6.876633 -1.096000   6.274300   90.487545   6.805365   
NorESM2-LM       3.755890 -0.445110   8.438124  135.482335   2.443470   
NorESM2-MM       7.175134 -1.420644   5.050620  100.678503   4.542638   
SAM0-UNICON      7.617295 -1.012835   7.520765   96.692747   6.093820   
UKESM1-0-LL      7.174172 -0.664608  10.794594   73.446598   7.417060   

Model                          EBM-epsilon                                   \
Parameter        gamma_2l  eff         q4x      lamg        t4x       cdeep   
Dataset                                                                       
ACCESS-CM2       0.615376  1.0    7.733057 -0.699488  11.055306   93.230040   
ACCESS-ESM1-5    0.722585  1.0    6.745166 -0.712660   9.464774   95.361976   
AWI-CM-1-1-MR    0.566624  1.0    8.168461 -1.209335   6.754507   56.493956   
BCC-CSM2-MR      0.973116  1.0    7.217011 -1.143931   6.308958   64.573536   
BCC-ESM1         0.584998  1.0    6.488241 -0.892194   7.272228   97.662585   
CAMS-CSM1-0      0.547402  1.0    9.076223 -1.917556   4.733225   56.972380   
CESM2            0.827251  1.0    8.478362 -0.658886  12.867720   75.909823   
CESM2-FV2        0.850765  1.0    7.707721 -0.581514  13.254569   92.727103   
CESM2-WACCM      0.813338  1.0    7.856972 -0.705814  11.131791   89.669971   
CESM2-WACCM-FV2  0.797907  1.0    7.011729 -0.601681  11.653568  112.097270   
CNRM-CM6-1       0.507962  1.0    7.305305 -0.751222   9.724564  145.225145   
CNRM-CM6-1-HR    0.506227  1.0    7.523272 -0.940570   7.998632   96.370197   
CNRM-ESM2-1      0.587744  1.0    5.7974

In [9]:
df_parms_calib = {'ar6': df['EBM-epsilon']}

## Smith et al. (2021, JGR)

In [10]:
owner = 'chrisroadmap'
repo = 'aerosol-history'
repo_s21 = RetrieveGitHub(owner, repo, './datain')

In [11]:
path = repo_s21.retrieve('data_input/scmpy2L_calib_n=44_eps=fit_v20200702.txt')

[2024-07-05 14:09:19 src.util] INFO:Use local file datain/chrisroadmap/aerosol-history/data_input/scmpy2L_calib_n=44_eps=fit_v20200702.txt retrieved from https://github.com/chrisroadmap/aerosol-history/raw/main/data_input/scmpy2L_calib_n%3D44_eps%3Dfit_v20200702.txt on 2024-06-11


In [12]:
df = (
    pd.read_table(path, delim_whitespace=True)
    .set_index('Model')
    .rename_axis(index='Dataset')
    .rename_axis(columns='Parameter')
    .rename(columns=str.lower)
    .rename(columns={
        'f4x': 'q4x',
        'lambda': 'lamg',
        'gamma': 'gamma_2l',
        'epsilon': 'eff',
    })
)
df

Parameter,q4x,lamg,cmix,cdeep,gamma_2l,eff
Dataset,,,,,,
ACCESS-CM2,7.6581,0.6877,8.8173,97.4564,0.5277,1.4939
ACCESS-ESM1-5,6.9731,0.7214,9.0163,96.7855,0.6054,1.7062
AWI-CM-1-1-MR,8.4080,1.2981,8.1684,54.6985,0.4933,1.3012
BCC-CSM2-MR,6.8870,1.0601,8.5130,73.6879,0.6415,1.3189
BCC-ESM1,6.6794,0.9350,8.4652,91.7194,0.5821,1.3315
CAMS-CSM1-0,8.8829,1.8751,10.0141,62.4085,0.5306,1.3362
CAS-ESM2-0,7.1346,0.9282,7.5720,72.8257,0.4498,1.4289
CESM2-FV2,7.9362,0.5592,7.9625,91.0971,0.6983,1.9094
CESM2-WACCM-FV2,7.1276,0.5936,7.6034,111.7078,0.7052,1.5395


In [13]:
df_parms_calib['s21'] = df

## MCE-2l

In [14]:
df_cmip6_norm1 = (
    pd.read_csv('./dataout/cmip6_normalized_1.csv', index_col=[0, 1, 2])
    .rename(columns=float)
)
df_cmip6_norm1

0.5       1.5       2.5       3.5    \
ACCESS-CM2  rtnt 1pctCO2       0.114668 -0.261170  0.364576  0.088614   
                 abrupt-4xCO2  7.384659  6.016853  5.279721  4.393673   
                 piControl    -0.045634  0.069533  0.033489 -0.003574   
            tas  1pctCO2       0.054810  0.026279 -0.027704  0.069290   
                 abrupt-4xCO2  1.077539  2.020234  2.329260  2.751447   
...                                 ...       ...       ...       ...   
UKESM1-0-LL rtnt abrupt-4xCO2  6.642358  5.841489  6.081244  5.399344   
                 piControl    -0.389239 -0.103244  0.566915 -0.098268   
            tas  1pctCO2       0.278635  0.140545  0.139373  0.124382   
                 abrupt-4xCO2  1.592799  2.120007  2.477257  3.003281   
                 piControl     0.356997  0.102347  0.117881  0.199185   

                                  4.5       5.5       6.5       7.5    \
ACCESS-CM2  rtnt 1pctCO2      -0.240797  0.032865  0.120728 -0.226226   
                 abrupt-4xCO2  5.090981  4.623253  4.417217  3.851686   
                 piControl    -0.040072 -0.041262 -0.376436 -0.301819   
            tas  1pctCO2       0.099759 -0.006149  0.038062  0.149160   
                 abrupt-4xCO2  2.818698  3.189038  3.482845  3.655235   
...                                 ...       ...       ...       ...   
UKESM1-0-LL rtnt abrupt-4xCO2  4.894980  4.545263  4.808677  4.775186   
                 piControl     0.044639 -0.024439 -0.189297  0.313945   
            tas  1pctCO2       0.088125  0.209777  0.235321  0.422771   
                 abrupt-4xCO2  3.480784  3.627754  3.621646  3.835658   
                 piControl     0.099007  0.139270  0.119387 -0.053199   

                                  8.5       9.5    ...  357.5  358.5  359.5  \
ACCESS-CM2  rtnt 1pctCO2       0.477206  0.306030  ...    NaN    NaN    NaN   
                 abrupt-4xCO2  4.005168  3.914475  ...    NaN    NaN    NaN   
                 piControl     0.061253 -0.441146  ...    NaN    NaN    NaN   
            tas  1pctCO2       0.034585  0.050589  ...    NaN    NaN    NaN   
                 abrupt-4xCO2  3.620542  3.825483  ...    NaN    NaN    NaN   
...                                 ...       ...  ...    ...    ...    ...   
UKESM1-0-LL rtnt abrupt-4xCO2  4.019885  4.107933  ...    NaN    NaN    NaN   
                 piControl    -0.125782  0.195047  ...    NaN    NaN    NaN   
            tas  1pctCO2       0.364995  0.319772  ...    NaN    NaN    NaN   
                 abrupt-4xCO2  4.241149  4.431383  ...    NaN    NaN    NaN   
                 piControl    -0.061748 -0.083119  ...    NaN    NaN    NaN   

                               360.5  361.5  362.5  363.5  364.5  365.5  366.5  
ACCESS-CM2  rtnt 1pctCO2         NaN    NaN    NaN    NaN    NaN    NaN    NaN  
                 abrupt-4xCO2    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
                 piControl       NaN    NaN    NaN    NaN    NaN    NaN    NaN  
            tas  1pctCO2         NaN    NaN    NaN    NaN    NaN    NaN    NaN  
                 abrupt-4xCO2    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
...                              ...    ...    ...    ...    ...    ...    ...  
UKESM1-0-LL rtnt abrupt-4xCO2    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
                 piControl       NaN    NaN    NaN    NaN    NaN    NaN    NaN  
            tas  1pctCO2         NaN    NaN    NaN    NaN    NaN    NaN    NaN  
                 abrupt-4xCO2    NaN    NaN    NaN    NaN    NaN    NaN    NaN  
                 piControl       NaN    NaN    NaN    NaN    NaN    NaN    NaN  

[266 rows x 367 columns]

In [15]:
def wrap_parm_estimate(df):
    """
    Wrapper function of ParmEstimate for a two-layer model

    Parameters
    ----------
    df
        Normalized GCM time series

    Returns
    -------
        Two-layer model parameters
    """
    time = df.columns.values[:150]

    # 1pctCO2 is shorter 
    data_gcm = [
        df.loc[('rtnt', 'abrupt-4xCO2')].dropna().values,
        df.loc[('tas', 'abrupt-4xCO2')].dropna().values,
        df.loc[('rtnt', '1pctCO2')].dropna().values,
        df.loc[('tas', '1pctCO2')].dropna().values,
    ]
    if [len(d1) for d1 in data_gcm] != [150, 150, 140, 140]:
        raise MCEExecError('Invalid data length')

    data_std = [
        df.loc[('rtnt', 'piControl')].dropna().std(),
        df.loc[('tas', 'piControl')].dropna().std(),
    ]
        
    obj = ParmEstimate(nl=2)
    args = ([1.], [2., 100.])
    kw = {'lamb': 1., 'ts4xeq': 7., 'beta': 1.}
    px = obj.initpars(*args, **kw)
    alpha, beta, lamb, asj, tauj = obj.minimize_wrap(time, data_gcm, data_std)

    return pd.Series({
        'q2x': alpha * np.log(2),
        'q4x': alpha * np.log(4) * beta,
        'co2_beta': beta,
        'lamg': lamb,
        **{
            f'tau{i}': value
            for i, value in enumerate(tauj)
        },
        **{
            f'a{i}': value
            for i, value in enumerate(asj)
        }
    }).rename_axis('Parameter')

In [16]:
mi = pd.MultiIndex.from_product([['rtnt', 'tas'], ['abrupt-4xCO2', '1pctCO2']])
datasets = sorted(pd.unique(df_cmip6_norm1.index.get_level_values(0)), key=str.lower)
dfout = {}

for dataset in datasets:
    dfin = df_cmip6_norm1.loc[dataset]
    if sum([k in dfin.index for k in mi]) != 4:
        continue
    dfout[dataset] = wrap_parm_estimate(dfin)

In [17]:
df_parms_calib['mce-2l'] = pd.concat(dfout, names=['Dataset']).unstack()
df_parms_calib['mce-2l']

Parameter,q2x,q4x,co2_beta,lamg,tau0,tau1,a0,a1
Dataset,,,,,,,,
ACCESS-CM2,3.082176,6.878696,1.115883,0.739969,5.547734,207.052268,0.492697,0.507303
ACCESS-ESM1-5,2.773499,5.800981,1.045788,0.762439,4.412132,207.278210,0.473682,0.526318
AWI-CM-1-1-MR,3.422610,7.292392,1.065326,1.160470,3.304301,118.719508,0.619379,0.380621
BCC-CSM2-MR,2.876654,5.994669,1.041952,0.979157,3.831630,162.568246,0.550478,0.449522
BCC-ESM1,2.748787,6.022407,1.095466,0.923221,4.202747,187.756843,0.545536,0.454464
CAMS-CSM1-0,4.020675,7.973425,0.991553,1.732842,2.652941,107.446834,0.700195,0.299805
CanESM5,3.317698,7.283649,1.097696,0.646104,6.174742,234.359651,0.497892,0.502108
CESM2,2.596173,6.487284,1.249394,0.628114,4.125655,202.255568,0.411640,0.588360
CESM2-FV2,2.455037,5.681416,1.157094,0.552648,3.651311,257.489836,0.382600,0.617400


## Hormonize the three calibrations

In [18]:
# Assumed factor of q2x to q4x based on AR6 WGI Table 7.2
factor_2x = 0.476

In [19]:
df = df_parms_calib['ar6'].copy()

df['lamg'] = -df['lamg']
df = pd.concat([df, ebm_to_irm(df)], axis=1)
df['q2x'] = df['q4x'] * factor_2x

df_parms_calib['ar6'] = df.drop('t4x', axis=1)

In [20]:
df = df_parms_calib['s21']

df = pd.concat([df, ebm_to_irm(df)], axis=1)
df['q2x'] = df['q4x'] * factor_2x

df_parms_calib['s21'] = df

In [21]:
df = df_parms_calib['mce-2l']

df = df.reindex(columns=df.columns.to_list() + ['eff'])
df = pd.concat([df, irm_to_ebm(df)], axis=1)

df_parms_calib['mce-2l'] = df

In [22]:
df_parms_calib = pd.concat(df_parms_calib, names=['Method'])
add_ecs_tcr(df_parms_calib)

In [23]:
df_parms_calib.to_csv('./dataout/parms_calib.csv')